## SIMPLE CHAR-RNN 

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(0)  
print ("TENSORFLOW VERSION IS %s" % (tf.__version__))

TENSORFLOW VERSION IS 1.0.1


## DEFINE TRAINING SEQUENCE

In [2]:
sentence = "If you want to build a ship, don't drum up people to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea. A designer knows he has achieved perfection not when there is nothing left to add, but when there is nothing left to take away. When you give yourself, you receive more than you give. A rock pile ceases to be a rock pile the moment a single man contemplates it, bearing within him the image of a cathedral. For true love is inexhaustible; the more you give, the more you have. And if you go to draw at the true fountainhead, the more water you draw, the more abundant is its flow. Night, the beloved. Night, when words fade and things come alive. When the destructive analysis of day is done, and all that is truly important becomes whole and sound again. When man reassembles his fragmentary self and grows with the calm of a tree. Transport of the mails, transport of the human voice, transport of flickering pictures-in this century as in others our highest accomplishments still have the single aim of bringing men together. It is only with the heart that one can see rightly; what is essential is invisible to the eye. He who has gone, so we but cherish his memory, abides with us, more potent, nay, more present than the living man. Grown-ups never understand anything for themselves, and it is tiresome for children to be always and forever explaining things to them. True happiness comes from the joy of deeds well done, the zest of creating things new. Each man must look to himself to teach him the meaning of life. It is not something discovered: it is something molded. Love does not consist in gazing at each other, but in looking outward together in the same direction. No single event can awaken within us a stranger totally unknown to us. To live is to be slowly born. It would be a bit too easy if we could go about borrowing ready-made souls. Only the unknown frightens men. But once a man has faced the unknown, that terror becomes the known. What makes the desert beautiful is that somewhere it hides a well. Perfection is achieved, not when there is nothing more to add, but when there is nothing left to take away. I know but one freedom, and that is the freedom of the mind. You become responsible, forever, for what you have tamed. Only he can understand what a farm is, what a country is, who shall have sacrificed part of himself to his farm or country, fought to save it, struggled to make it beautiful. Only then will the love of farm or country fill his heart. And now here is my secret, a very simple secret: it is only with the heart that one can see rightly, what is essential is invisible to the eye. A chief is a man who assumes responsibility. He says 'I was beaten', he does not say 'My men were beaten.' A civilization is a heritage of beliefs, customs, and knowledge slowly accumulated in the course of centuries, elements difficult at times to justify by logic, but justifying themselves as paths when they lead somewhere, since they open up for man his inner distance. We say nothing essential about the cathedral when we speak of its stones. We say nothing essential about Man when we seek to define him by the qualities of men. How could there be any question of acquiring or possessing, when the one thing needful for a man is to become - to be at last, and to die in the fullness of his being."
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)

FOLLOWING IS OUR TRAINING SEQUENCE:
If you want to build a ship, don't drum up people to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea. A designer knows he has achieved perfection not when there is nothing left to add, but when there is nothing left to take away. When you give yourself, you receive more than you give. A rock pile ceases to be a rock pile the moment a single man contemplates it, bearing within him the image of a cathedral. For true love is inexhaustible; the more you give, the more you have. And if you go to draw at the true fountainhead, the more water you draw, the more abundant is its flow. Night, the beloved. Night, when words fade and things come alive. When the destructive analysis of day is done, and all that is truly important becomes whole and sound again. When man reassembles his fragmentary self and grows with the calm of a tree. Transport of the mails, transport of the human voice, trans

## DEFINE VOCABULARY AND DICTIONARY

In [3]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
print ("VOCABULARY: ")
print (char_set)
print ("DICTIONARY: ")
print (char_dic)

VOCABULARY: 
[' ', "'", '-', ',', '.', ';', ':', 'A', 'B', 'E', 'G', 'F', 'I', 'H', 'M', 'L', 'O', 'N', 'P', 'T', 'W', 'Y', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']
DICTIONARY: 
{' ': 0, "'": 1, '-': 2, ',': 3, '.': 4, ';': 5, ':': 6, 'A': 7, 'B': 8, 'E': 9, 'G': 10, 'F': 11, 'I': 12, 'H': 13, 'M': 14, 'L': 15, 'O': 16, 'N': 17, 'P': 18, 'T': 19, 'W': 20, 'Y': 21, 'a': 22, 'c': 23, 'b': 24, 'e': 25, 'd': 26, 'g': 27, 'f': 28, 'i': 29, 'h': 30, 'k': 31, 'j': 32, 'm': 33, 'l': 34, 'o': 35, 'n': 36, 'q': 37, 'p': 38, 's': 39, 'r': 40, 'u': 41, 't': 42, 'w': 43, 'v': 44, 'y': 45, 'x': 46, 'z': 47}


## CONFIGURE NETWORK

In [4]:
data_dim    = len(char_set)
num_classes = len(char_set)
hidden_size     = 64
sequence_length = 10  # Any arbitrary number

## SET TRAINING BATCHES

In [5]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index
    dataX.append(x)
    dataY.append(y)
    if i < 10:
        print ("[%4d/%4d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*12, x, y))

[   0/3409] [If you wan]=>[f you want]
            [12, 28, 0, 45, 35, 41, 0, 43, 22, 36]=>[28, 0, 45, 35, 41, 0, 43, 22, 36, 42]
[   1/3409] [f you want]=>[ you want ]
            [28, 0, 45, 35, 41, 0, 43, 22, 36, 42]=>[0, 45, 35, 41, 0, 43, 22, 36, 42, 0]
[   2/3409] [ you want ]=>[you want t]
            [0, 45, 35, 41, 0, 43, 22, 36, 42, 0]=>[45, 35, 41, 0, 43, 22, 36, 42, 0, 42]
[   3/3409] [you want t]=>[ou want to]
            [45, 35, 41, 0, 43, 22, 36, 42, 0, 42]=>[35, 41, 0, 43, 22, 36, 42, 0, 42, 35]
[   4/3409] [ou want to]=>[u want to ]
            [35, 41, 0, 43, 22, 36, 42, 0, 42, 35]=>[41, 0, 43, 22, 36, 42, 0, 42, 35, 0]
[   5/3409] [u want to ]=>[ want to b]
            [41, 0, 43, 22, 36, 42, 0, 42, 35, 0]=>[0, 43, 22, 36, 42, 0, 42, 35, 0, 24]
[   6/3409] [ want to b]=>[want to bu]
            [0, 43, 22, 36, 42, 0, 42, 35, 0, 24]=>[43, 22, 36, 42, 0, 42, 35, 0, 24, 41]
[   7/3409] [want to bu]=>[ant to bui]
            [43, 22, 36, 42, 0, 42, 35, 0, 24, 41]=>[22, 

In [6]:
ndata      = len(dataX)
batch_size = 512
print ("     'NDATA' IS %d" % (ndata))
print ("'BATCH_SIZE' IS %d" % (batch_size))

     'NDATA' IS 3399
'BATCH_SIZE' IS 512


## DEFINE PLACEHOLDERS

In [7]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])
X_OH = tf.one_hot(X, num_classes)
print ("'sequence_length' IS [%d]" % (sequence_length))
print ("    'num_classes' IS [%d]" % (num_classes))
print("'X' LOOKS LIKE \n   [%s]" % (X))  
print("'X_OH' LOOKS LIKE \n   [%s]" % (X_OH))

'sequence_length' IS [10]
    'num_classes' IS [48]
'X' LOOKS LIKE 
   [Tensor("Placeholder:0", shape=(?, 10), dtype=int32)]
'X_OH' LOOKS LIKE 
   [Tensor("one_hot:0", shape=(?, 10, 48), dtype=float32)]


## DEFINE MODEL

In [8]:
num_hidden = 256
with tf.variable_scope('CHAR-RNN', reuse=False):
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _hiddens  = tf.contrib.layers.fully_connected(X_OH, num_hidden, activation_fn=tf.nn.relu)
    _outputs, _states = tf.nn.dynamic_rnn(cell, _hiddens, dtype=tf.float32)
    _outputs  = tf.contrib.layers.fully_connected(_outputs, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs = tf.reshape(_outputs, [batch_size, sequence_length, num_classes])
print ("OUTPUTS LOOKS LIKE [%s]" % (outputs))
print ("MODEL DEFINED.")

OUTPUTS LOOKS LIKE [Tensor("CHAR-RNN/Reshape:0", shape=(512, 10, 48), dtype=float32)]
MODEL DEFINED.


## DEFINE TF FUNCTIONS

In [9]:
weights = tf.ones([batch_size, sequence_length]) # EQUAL WEIGHTS
seq_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss  = tf.reduce_mean(seq_loss)
optm  = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
print ("FUNCTIONS DEFINED.")

FUNCTIONS DEFINED.


## OPTIMIZE

In [10]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
MAXITER = 20000
for i in range(MAXITER):
    randidx = np.random.randint(low=0, high=ndata, size=batch_size)
    batchX = [dataX[iii] for iii in randidx]
    batchY = [dataY[iii] for iii in randidx]
    feeds = {X: batchX, Y: batchY}
    _, loss_val, results = sess.run(
        [optm, loss, outputs], feed_dict=feeds)
    if (i%1000) == 0:
        print ("[%5d] loss_val: %.5f " % (i, loss_val))

[    0] loss_val: 3.87109 
[ 1000] loss_val: 1.69164 
[ 2000] loss_val: 1.08516 
[ 3000] loss_val: 0.77485 
[ 4000] loss_val: 0.65322 
[ 5000] loss_val: 0.59381 
[ 6000] loss_val: 0.57462 
[ 7000] loss_val: 0.54960 
[ 8000] loss_val: 0.54696 
[ 9000] loss_val: 0.53584 
[10000] loss_val: 0.52734 
[11000] loss_val: 0.53316 
[12000] loss_val: 0.53226 
[13000] loss_val: 0.53119 
[14000] loss_val: 0.52793 
[15000] loss_val: 0.53143 
[16000] loss_val: 0.52363 
[17000] loss_val: 0.52421 
[18000] loss_val: 0.51945 
[19000] loss_val: 0.52086 


## PRINT CHARS

In [11]:
randidx = np.random.randint(low=0, high=ndata, size=batch_size)
batchX = [dataX[iii] for iii in randidx]
batchY = [dataY[iii] for iii in randidx]
feeds = {X: batchX, Y: batchY}
results = sess.run(outputs, feed_dict=feeds)
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    chars = [char_set[t] for t in index]
    if j < 10:
        print (chars)

['h', ' ', ' ', 'e', 'a', 's', 'y', ' ', 'i', 'f']
[' ', ' ', 'o', 'r', ' ', 'b', 'e', 'c', 'o', 'm']
['e', 'n', ' ', 's', ' ', 'd', 't', 'i', 'l', 'l']
[' ', ' ', 'W', 'h', 'e', 't', ' ', 'm', 'a', 'k']
['t', 'f', ' ', 't', 'e', 'l', 'i', 'e', 'f', 's']
['t', 't', ' ', 'T', 'o', ' ', 'l', 'i', 'v', 'e']
['n', 'e', 'i', 'n', '.', ' ', 'W', 'h', 'e', 'n']
['t', ' ', 'd', ' ', 't', 'o', 'r', 'k', ',', ' ']
[' ', 'n', 'o', 'f', ' ', 'c', 'e', 'n', 't', 'u']
['e', 'n', 'h', 't', 'l', 'y', ';', ' ', 'w', 'h']


### SAMPLING FUNCTION 

In [12]:
LEN = 1;
# XL = tf.placeholder(tf.int32, [None, LEN])
XL     = tf.placeholder(tf.int32, [None, 1])
XL_OH  = tf.one_hot(XL, num_classes)
with tf.variable_scope('CHAR-RNN', reuse=True):
    cell_L = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell_L = rnn.MultiRNNCell([cell_L] * 2, state_is_tuple=True)
    istate = cell_L.zero_state(batch_size=1, dtype=tf.float32)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _hiddens  = tf.contrib.layers.fully_connected(XL_OH, num_hidden, activation_fn=tf.nn.tanh)
    _outputs_L, states_L = tf.nn.dynamic_rnn(cell_L, _hiddens
                                , initial_state=istate, dtype=tf.float32)
    _outputs_L  = tf.contrib.layers.fully_connected(
        _outputs_L, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs_L = tf.reshape(_outputs_L, [LEN, 1, num_classes])
print (XL)

def weighted_pick(weights):
    t = np.cumsum(weights)
    s = np.sum(weights)
    return(int(np.searchsorted(t, np.random.rand(1)*s)))
def softmax(x):
    alpha = 2
    e_x = np.exp(alpha*(x - np.max(x)))
    return e_x / np.sum(e_x) # only difference

Tensor("Placeholder_2:0", shape=(?, 1), dtype=int32)


## SAMPLE

### BURNIN

In [27]:
prime = "Perfection"
istateval = sess.run(cell_L.zero_state(1, tf.float32))
for c in prime[:-1]:
    index = char_dic[c]
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L]
                        , feed_dict={XL:inval, istate:istateval})
    istateval = stateval

### SAMPLE

In [28]:
inval  = [[char_dic[prime[-1]]]]
outval, stateval = sess.run([outputs_L, states_L]
                    , feed_dict={XL:inval, istate:istateval})
istateval = stateval
index = np.argmax(outval)
char  = char_set[index]
chars = ''
for i in range(1000):
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L]
                        , feed_dict={XL:inval, istate:istateval})
    istateval = stateval
    # index = np.argmax(outval)
    index = weighted_pick(softmax(outval))
    char  = char_set[index]
    chars += char
print ("SAMPLED SETENCE: \n %s" % (prime+chars))
print ("\nORIGINAL SENTENCE: \n %s" % (sentence))

SAMPLED SETENCE: 
 Perfectionis acquiring or pound again. When man reassembles his frather in the desert beautiful is invisible to take away. I know but cherish his mem born. It when there it hid, doredkn-in this centuries, elemen. How could there be anything for the endledss of men. How could there be anything for the endlates it, bearing men togle aim of bringing men together in the desert beautiful. Only he does not say 'Ma ring things to the eye. A chief is its flow. Night, the more than the live you go to the eye. A chief is its flow. Night, the more than the live yourself, you receive more than you give. A rock of his beinly to be always and forever explaining things to them. True happiness comes froms, and knowledge slowly born. It whaus him the meaning of life. It is not something molded. Love does not consist in gazing at each him the meaning of aim or possessing, when you give. A rock of his beinly to be always accumulating things to the eye. A chief is invisible the fullness